In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/23 20:16:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/23 20:16:19 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/02/23 20:16:19 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/02/23 20:16:19 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [4]:
df_green = spark.read.parquet('/home/pedroh/dataengineer-zoomcamp2024/week6/code/data/pq/green/*/*')

In [5]:
df_green = df_green \
    .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

In [8]:
df_yellow = spark.read.parquet('/home/pedroh/dataengineer-zoomcamp2024/week6/code/data/pq/yellow/*/*')

In [9]:
df_yellow = df_yellow \
    .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

In [12]:
common_columns = []

yellow_columns = set(df_yellow.columns)

for col in df_green.columns:
    if col in yellow_columns:
        common_columns.append(col)

In [13]:
common_columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'congestion_surcharge']

In [15]:
from pyspark.sql import functions as F

In [19]:
df_green_final = df_green.select(common_columns).withColumn('service_type', F.lit('Green'))

In [17]:
df_yellow_final = df_yellow.select(common_columns).withColumn('service_type', F.lit('Yellow'))

In [20]:
df_trips_data = df_green_final.unionAll(df_yellow_final)

In [22]:
df_trips_data.groupBy('service_type').count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       Green| 2304517|
|      Yellow|39649199|
+------------+--------+



In [24]:
df_trips_data.createOrReplaceTempView('trips_data')

In [37]:
 spark.sql("""
             SELECT service_type, count(*) FROM trips_data group by 1 LIMIT 10;
 """).show()

+------------+--------+
|service_type|count(1)|
+------------+--------+
|       Green| 2304517|
|      Yellow|39649199|
+------------+--------+



In [38]:
df_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
     PULocationID AS revenue_zone,
     date_trunc('month', pickup_datetime) AS revenue_month,
     service_type, 

    -- Revenue calculation 
     ROUND(SUM(fare_amount),2) AS revenue_monthly_fare,
     ROUND(SUM(extra),2) AS revenue_monthly_extra,
     ROUND(SUM(mta_tax),2) AS revenue_monthly_mta_tax,
     ROUND(SUM(tip_amount),2) AS revenue_monthly_tip_amount,
     ROUND(SUM(tolls_amount),2) AS revenue_monthly_tolls_amount,
     ROUND(SUM(improvement_surcharge),2) AS revenue_monthly_improvement_surcharge,
     ROUND(SUM(total_amount),2) AS revenue_monthly_total_amount,
     ROUND(SUM(congestion_surcharge),2) AS revenue_monthly_congestion_surcharge,

    -- Additional calculations
     ROUND(AVG(passenger_count),2) AS avg_montly_passenger_count,
     ROUND(AVG(trip_distance),2) AS avg_montly_trip_distance
FROM
    trips_data
GROUP BY
    1, 2, 3
""")

In [40]:
df_result.groupBy('service_type').count().show()

+------------+-----+
|service_type|count|
+------------+-----+
|       Green| 4792|
|      Yellow| 5103|
+------------+-----+



In [39]:
df_result.write.mode("overwrite").parquet('/home/pedroh/dataengineer-zoomcamp2024/week6/code/data/report/revenue/')